In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy.stats import zscore, linregress
from pandas.plotting import scatter_matrix
from scipy import stats
import numpy as np

In [21]:
# Function to remove outliers from a dataframe
from sklearn.neighbors import LocalOutlierFactor

def remove_outliers(df):
    # Initialize a boolean mask to keep track of rows to drop
    outlier_rows_mask = np.zeros(len(df), dtype=bool)

    # Iterate over each column
    for col in df.columns:
        # Fit the LocalOutlierFactor model to the column data
        lof = LocalOutlierFactor()
        outliers = lof.fit_predict(df[col].values.reshape(-1, 1))

        # Mark rows with outliers in this column
        outlier_rows_mask = np.logical_or(outlier_rows_mask, outliers == -1)

    # Drop rows with outliers
    # cleaned_df = df[~outlier_rows_mask]
    return outlier_rows_mask

In [2]:
fluxnet_info = pd.read_csv("../data/EC/fluxnet/sites_info.csv")
ameriflux_info = pd.read_csv("../data/EC/Ameriflux/sites_info.tsv", delimiter="\t")

In [3]:
fluxnet_names = fluxnet_info["ID"].to_list()
fluxnet_types = fluxnet_info["type"].to_list()
ameriflux_names = ameriflux_info["Site ID"].to_list()
ameriflux_types = ameriflux_info["Vegetation Abbreviation (IGBP)"].to_list()

In [8]:
combined_names = list(set(ameriflux_names + fluxnet_names))
combined_types = []
for name in combined_names:
    if name in ameriflux_names and name in fluxnet_names:
        # Choose a type from either fluxnet_types or ameriflux_types
        combined_types.append(fluxnet_types[fluxnet_names.index(name)])
    elif name in ameriflux_names:
        combined_types.append(ameriflux_types[ameriflux_names.index(name)])
    else:
        combined_types.append(fluxnet_types[fluxnet_names.index(name)])

In [12]:
dfs = []

# Iterate over the range of length of 'name' column in sites_info
for i in range(len(combined_names)):
    site_name = combined_names[i]
    site_type = combined_types[i]

    # Set the file path based on whether the combined name is in ameriflux_names or fluxnet_names
    if site_name in ameriflux_names:
        file = glob.glob("../data/EC/Ameriflux/AMF_" + site_name + "*DD*")
    else:
        file = glob.glob("../data/EC/fluxnet/FLX_" + site_name + "*DD*")

    # Open the CSV file
    ec = pd.read_csv(file[0])
    ec.loc[:, "type"] = site_type
    ec.loc[:, "name"] = site_name

    ec["t"] = pd.to_datetime(ec["TIMESTAMP"], format="%Y%m%d")
    ec = ec.set_index("t")

    # Append the DataFrame to the list
    dfs.append(ec)

# Concatenate all DataFrames in the list
combined_ec = pd.concat(dfs)

Combine all the satellite data


In [16]:
MCD43_fluxnet = []
MCD15_fluxnet = []
MCD43_ameriflux = []
MCD15_ameriflux = []

# Loop over batches (#5) of downloaded data
for i in range(1, 5):
    refl_fluxnet = glob.glob(
        "../data/EC/fluxnet/sat_data/*batch" + str(i) + "*MCD43A4-061-results.csv"
    )
    sat_refl_fluxnet = pd.read_csv(refl_fluxnet[0])
    sat_refl_fluxnet.loc[:, "time"] = pd.to_datetime(sat_refl_fluxnet["Date"])
    sat_refl_fluxnet.set_index(sat_refl_fluxnet["Date"], inplace=True)
    MCD43_fluxnet.append(sat_refl_fluxnet)

    fpar_fluxnet = glob.glob(
        "../data/EC/fluxnet/sat_data/*batch" + str(i) + "*MCD15A3H-061-results.csv"
    )
    sat_fpar_fluxnet = pd.read_csv(fpar_fluxnet[0])
    sat_fpar_fluxnet.loc[:, "time"] = pd.to_datetime(sat_fpar_fluxnet["Date"])
    sat_fpar_fluxnet.set_index(sat_fpar_fluxnet["Date"], inplace=True)
    MCD15_fluxnet.append(sat_fpar_fluxnet)

    if i < 5:
        refl_ameriflux = glob.glob(
            "../data/EC/Ameriflux/sat_data/*batch" + str(i) + "*MCD43A4-061-results.csv"
        )

        sat_refl_ameriflux = pd.read_csv(refl_ameriflux[0])
        sat_refl_ameriflux.loc[:, "time"] = pd.to_datetime(sat_refl_ameriflux["Date"])
        sat_refl_ameriflux.set_index(sat_refl_ameriflux["Date"], inplace=True)
        MCD43_ameriflux.append(sat_refl_ameriflux)

        fpar_ameriflux = glob.glob(
            "../data/EC/Ameriflux/sat_data/*batch"
            + str(i)
            + "*MCD15A3H-061-results.csv"
        )
        fpar_ameriflux = pd.read_csv(fpar_ameriflux[0])

        fpar_ameriflux.loc[:, "time"] = pd.to_datetime(fpar_ameriflux["Date"])
        fpar_ameriflux.set_index(fpar_ameriflux["Date"], inplace=True)
        MCD15_ameriflux.append(fpar_ameriflux)


refl_fluxnet = pd.concat(MCD43_fluxnet)
refl_fluxnet = refl_fluxnet.rename(columns={"ID": "name"})

fpar_fluxnet = pd.concat(MCD15_fluxnet)
fpar_fluxnet = fpar_fluxnet.rename(columns={"ID": "name"})

refl_ameriflux = pd.concat(MCD43_ameriflux)
refl_ameriflux = refl_ameriflux.rename(columns={"ID": "name"})
fpar_ameriflux = pd.concat(MCD15_ameriflux)
fpar_ameriflux = fpar_ameriflux.rename(columns={"ID": "name"})

In [19]:
combined_refl = []
combined_fpar = []

for name in combined_names:
    if name in ameriflux_names:
        selected_refl = refl_ameriflux[refl_ameriflux["name"] == name]
        selected_fpar = fpar_ameriflux[fpar_ameriflux["name"] == name]
    else:
        selected_refl = refl_fluxnet[refl_fluxnet["name"] == name]
        selected_fpar = fpar_fluxnet[fpar_fluxnet["name"] == name]

    combined_refl.append(selected_refl)
    combined_fpar.append(selected_fpar)

combined_refl = pd.concat(combined_refl)
combined_fpar = pd.concat(combined_fpar)

In [ ]:
bad_sites_list = []
df_stat = []
df_stat_pft = []

for i in range(len(combined_names)):
    print(i)
    site_name = combined_names[i]
    site_type = combined_types[i]
    selected_ec = combined_ec[combined_ec["name"] == site_name]

    if selected_ec["PPFD_IN_QC"].isna().all():
        print("No PPFD data for " + site_name)
        bad_sites_list.append(site_name)
        # continue
    gpp = selected_ec[["GPP_NT_VUT_REF"]]
    par = selected_ec[["PPFD_IN"]]
    par_qc = selected_ec[["PPFD_IN_QC"]]
    ec_daily = pd.concat([gpp, par, par_qc], axis=1).rename(
        columns={"GPP_NT_VUT_REF": "gpp", "PPFD_IN": "par", "PPFD_IN_QC": "par_qc"}
    )
    ec_daily = ec_daily[ec_daily["par_qc"] == 1]
    ec_daily = ec_daily[ec_daily["gpp"] != -9999]

    # filter fpar based on QC flags
    filtered_fpar = selected_fpar[
        (selected_fpar["MCD15A3H_061_FparLai_QC_MODLAND"] == "0b0")
        & (selected_fpar["MCD15A3H_061_FparLai_QC_DeadDetector"] == "0b0")
        & (selected_fpar["MCD15A3H_061_FparLai_QC_CloudState"] == "0b00")
        & (selected_fpar["MCD15A3H_061_FparLai_QC_SCF_QC"].isin(["0b000", "0b001"]))
    ].copy()

    filtered_fpar.loc[:, "time"] = pd.to_datetime(filtered_fpar["Date"])
    fpar_4days = filtered_fpar[["MCD15A3H_061_Fpar_500m"]]
    fpar_4days.set_index(filtered_fpar["time"], inplace=True)
    fpar_daily = fpar_4days.resample("D").interpolate(method="linear")
    fpar_daily.rename(columns={"MCD15A3H_061_Fpar_500m": "fpar"}, inplace=True)
    if len(fpar_daily) < 30:
        print("Not enough fpar for" + site_name)
        bad_sites_list.append(site_name)
        continue
    fpar_daily

    # Now filter reflectance
    filtered_refl = selected_refl[
        (
            selected_refl[
                "MCD43A4_061_BRDF_Albedo_Band_Mandatory_Quality_Band1_MODLAND"
            ]
            == "0b000"
        )
        & (
            selected_refl[
                "MCD43A4_061_BRDF_Albedo_Band_Mandatory_Quality_Band2_MODLAND"
            ]
            == "0b000"
        )
    ].copy()
    filtered_refl.loc[:, "time"] = pd.to_datetime(filtered_refl["Date"])
    red_daily = filtered_refl[["MCD43A4_061_Nadir_Reflectance_Band1"]]
    nir_daily = filtered_refl[["MCD43A4_061_Nadir_Reflectance_Band2"]]

    refl_daily = pd.concat([red_daily, nir_daily], axis=1).rename(
        {
            "MCD43A4_061_Nadir_Reflectance_Band1": "red",
            "MCD43A4_061_Nadir_Reflectance_Band2": "nir",
        },
        axis=1,
    )
    refl_daily.set_index(filtered_refl["time"], inplace=True)
    daily_df = ec_daily.merge(refl_daily, left_index=True, right_index=True).merge(
        fpar_daily, left_index=True, right_index=True
    )
    daily_df.loc[:, "ndvi"] = (daily_df["nir"] - daily_df["red"]) / (
        daily_df["nir"] + daily_df["red"]
    )
    daily_df.loc[:, "nirv"] = daily_df["ndvi"] * daily_df["nir"]
    daily_df.loc[:, "nirvp"] = daily_df["nirv"] * daily_df["par"]
    daily_df.loc[:, "fesc"] = daily_df["nirv"] / daily_df["fpar"]
    daily_df.loc[:, "dasf"] = daily_df["nirv"] / 0.9789

    daily_df.loc[:, "lue"] = daily_df["gpp"] / (daily_df["par"] * daily_df["fpar"])
    daily_df = daily_df.replace([-np.inf, np.inf], np.nan).dropna()

    selected_ec_years = daily_df.index.year.unique()
    if len(selected_ec_years) < 3:
        print("Not enough years for " + site_name)
        bad_sites_list.append(site_name)
        continue
    # Assuming df is your DataFrame
    outlier_rows_mask = remove_outliers(daily_df)
    df_no_outliers = daily_df[~outlier_rows_mask].copy()
    df_stat_pft.append(site_type)

    # max_lue_index = df_no_outliers["lue"].idxmax()
    # df_stat.append(df_no_outliers.loc[max_lue_index])

    median_lue = df_no_outliers["lue"].median()
    median_lue_index = (df_no_outliers["lue"] - median_lue).abs().idxmin()
    df_stat.append(df_no_outliers.loc[median_lue_index])
    

    # smoothed_lue = df_no_outliers.lue.rolling(window=7).mean()
    # max_lue_index = smoothed_lue.groupby(smoothed_lue.index.year).idxmax()
    # max_lue_index.dropna(inplace=True)
    # df_stat.append(df_no_outliers.loc[max_lue_index].mean())
    

In [ ]:
df_stat = pd.concat(df_stat, axis=1).T
df_stat_pft = pd.DataFrame(df_stat_pft, index=df_stat.index)
df_stat

In [ ]:
outlier_index = remove_outliers(df_stat)
df_stat_clean = df_stat[~outlier_index].copy()
df_stat_pft_clean = df_stat_pft[~outlier_index]
df_stat_clean = df_stat_clean.assign(pft_clean=df_stat_pft_clean)

In [ ]:
# Do this later! 
# Define the bin edges
# bin_edges = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

# Bin the data based on fpar
# df_stat_clean["fpar_bin"] = pd.cut(df_stat_clean["fpar"], bins=bin_edges)
# df_stat_clean
# Select data in a specific bin
# df_bin = df_stat_clean[df_stat_clean["fpar_bin"] == pd.Interval(0.6, 0.8, closed="right")]


In [ ]:
import seaborn as sns

df_melted = pd.melt(df_stat_clean, id_vars="pft_clean", value_vars=["lue", "fesc"])

# Create a box plot
plt.figure(figsize=(10, 6))
sns.boxplot(x="pft_clean", y="value", hue="variable", data=df_melted)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a FacetGrid with each unique value of pft_clean
g = sns.FacetGrid(df_stat_clean, col="pft_clean", col_wrap=3)

# Map scatter plot to each subplot
g.map_dataframe(sns.scatterplot, x="lue", y="fesc")

# Set the labels and title
g.set_axis_labels("lue", "fesc")
g.fig.suptitle("Scatter Plot: lue vs fesc for each pft_clean", y=1.02)

# Show the plot
plt.show()


In [ ]:
import seaborn as sns

# Create a scatter plot with hue based on pft_clean
sns.scatterplot(data=df_stat_clean, x="lue", y="fesc", hue="pft_clean")

# Set the labels and title
plt.xlabel("lue")
plt.ylabel("fesc")
plt.title("Scatter Plot: lue vs fesc")

# Show the plot
plt.show()

In [ ]:
var_list = [
    "gpp",
    "lue",
    "fesc",
    "nirv",
    "nirvp",
    "dasf",
    "ndvi",
    "fpar",
    "par",
    "pft_clean",
]
df_mean = df_stat_clean[var_list].groupby("pft_clean").mean()
df_mean = df_mean.reset_index()
df_mean

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))

x = "lue"
y = "fpar"

scatter = sns.scatterplot(data=df_mean, x=x, y=y, hue="pft_clean", ax=ax[0])
for line in range(0, df_mean.shape[0]):
    scatter.text(
        df_mean[x][line] + 0.001,
        df_mean[y][line],
        df_mean.pft_clean[line],
        horizontalalignment="left",
        size="medium",
        color="black",
        weight="semibold",
    )

ax[0].set_xlabel(x)
ax[0].set_ylabel(y)
ax[0].set_title(x + " vs " + y)
scatter.legend_.remove()
x = "gpp"
y = "fesc"

scatter = sns.scatterplot(data=df_mean, x=x, y=y, hue="pft_clean", ax=ax[1])
for line in range(0, df_mean.shape[0]):
    scatter.text(
        df_mean[x][line] + 0.001,
        df_mean[y][line],
        df_mean.pft_clean[line],
        horizontalalignment="left",
        size="medium",
        color="black",
        weight="semibold",
    )

ax[1].set_xlabel(x)
ax[1].set_ylabel(y)
ax[1].set_title(x + " vs " + y)

# Remove the legend
scatter.legend_.remove()

plt.show()

In [ ]:
df_mean = df_mean.set_index("pft_clean")

In [ ]:
from pandas.plotting import scatter_matrix

axs = scatter_matrix(df_mean, figsize=(10, 10), alpha=1)
# Loop over the diagonal
for i in range(len(df_mean.columns)):
    for j in range(len(df_mean.columns)):
        if i != j:
            # Calculate the R^2 value
            r2 = (
                np.corrcoef(df_mean[df_mean.columns[i]], df_mean[df_mean.columns[j]])[
                    0, 1
                ]
                ** 2
            )
            # Add the R^2 value to the plot
            axs[i, j].annotate(
                "R^2 = {:.2f}".format(r2),
                (0.5, 0.8),
                xycoords="axes fraction",
                ha="center",
            )

### Below code are for time series plot of the data


In [ ]:
# i = 335 # US-Ha1
bad_sites = []

i = 186
site_name = combined_names[i]
site_type = combined_types[i]
print([site_name, site_type])
selected_ec = combined_ec[combined_ec["name"] == site_name]
selected_refl = combined_refl[combined_refl["name"] == site_name]
selected_fpar = combined_fpar[combined_fpar["name"] == site_name]

In [ ]:
if selected_ec["PPFD_IN_QC"].isna().all():
    print("No PPFD data for this site")
    bad_sites.append(site_name)
    # continue

gpp = selected_ec[["GPP_NT_VUT_REF"]]
par = selected_ec[["PPFD_IN"]]
par_qc = selected_ec[["PPFD_IN_QC"]]
ec_daily = pd.concat([gpp, par, par_qc], axis=1).rename(
    columns={"GPP_NT_VUT_REF": "gpp", "PPFD_IN": "par", "PPFD_IN_QC": "par_qc"}
)
ec_daily = ec_daily[ec_daily["par_qc"] == 1]
ec_daily = ec_daily[ec_daily["gpp"] != -9999]
if len(ec_daily) < 30:
    print("Not enough data for this site")
    bad_sites.append(site_name)
    # continue

ec_daily

In [ ]:
# filter fpar based on QC flags
filtered_fpar = selected_fpar[
    (selected_fpar["MCD15A3H_061_FparLai_QC_MODLAND"] == "0b0")
    & (selected_fpar["MCD15A3H_061_FparLai_QC_DeadDetector"] == "0b0")
    & (selected_fpar["MCD15A3H_061_FparLai_QC_CloudState"] == "0b00")
    & (selected_fpar["MCD15A3H_061_FparLai_QC_SCF_QC"].isin(["0b000", "0b001"]))
].copy()

filtered_fpar.loc[:, "time"] = pd.to_datetime(filtered_fpar["Date"])
fpar_4days = filtered_fpar[["MCD15A3H_061_Fpar_500m"]]
fpar_4days.set_index(filtered_fpar["time"], inplace=True)
fpar_daily = fpar_4days.resample("D").interpolate(method="linear")
fpar_daily.rename(columns={"MCD15A3H_061_Fpar_500m": "fpar"}, inplace=True)
if len(fpar_daily) < 30:
    print("Not enough fpar for" + site_name)
    bad_sites.append(site_name)
    # continue
fpar_daily

In [ ]:
# Now filter reflectance
filtered_refl = selected_refl[
    (
        selected_refl["MCD43A4_061_BRDF_Albedo_Band_Mandatory_Quality_Band1_MODLAND"]
        == "0b000"
    )
    & (
        selected_refl["MCD43A4_061_BRDF_Albedo_Band_Mandatory_Quality_Band2_MODLAND"]
        == "0b000"
    )
    & (
        selected_refl["MCD43A4_061_BRDF_Albedo_Band_Mandatory_Quality_Band4_MODLAND"]
        == "0b000"
    )
].copy()
filtered_refl.loc[:, "time"] = pd.to_datetime(filtered_refl["Date"])
red_daily = filtered_refl[["MCD43A4_061_Nadir_Reflectance_Band1"]]
nir_daily = filtered_refl[["MCD43A4_061_Nadir_Reflectance_Band2"]]
green_daily = filtered_refl[["MCD43A4_061_Nadir_Reflectance_Band4"]]

refl_daily = pd.concat([red_daily, nir_daily], axis=1).rename(
    {
        "MCD43A4_061_Nadir_Reflectance_Band1": "red",
        "MCD43A4_061_Nadir_Reflectance_Band2": "nir",
        "MCD43A4_061_Nadir_Reflectance_Band4": "green",
    },
    axis=1,
)
refl_daily.set_index(filtered_refl["time"], inplace=True)
refl_daily

In [ ]:
daily_df = ec_daily.merge(refl_daily, left_index=True, right_index=True).merge(
    fpar_daily, left_index=True, right_index=True
)
daily_df

In [ ]:
daily_df.loc[:, "ndvi"] = (daily_df["nir"] - daily_df["red"]) / (
    daily_df["nir"] + daily_df["red"]
)
daily_df.loc[:, "nirv"] = daily_df["ndvi"] * daily_df["nir"]
daily_df.loc[:, "nirvp"] = daily_df["nirv"] * daily_df["par"]
daily_df.loc[:, "fesc"] = daily_df["nirv"] / daily_df["fpar"]

daily_df.loc[:, "lue"] = daily_df["gpp"] / (daily_df["par"] * daily_df["fpar"])
daily_df = daily_df.replace([-np.inf, np.inf], np.nan).dropna()

In [ ]:
# Calculate Z-scores
z_scores = np.abs(zscore(daily_df))

# Set a threshold for outliers
threshold = 3

# Get a boolean mask where True indicates it is an outlier
outliers = (z_scores > threshold).any(axis=1)

# Remove outliers
daily_df_no_outliers = daily_df[~outliers].copy()
daily_df_no_outliers

In [ ]:
attributes = ["lue", "fesc", "nirv", "nirvp", "ndvi", "gpp"]
scatter_matrix(daily_df_no_outliers[attributes], figsize=(10, 10))
r2_values_daily = {}
attributes = ["lue", "fesc", "nirv", "nirvp", "ndvi", "gpp"]

for x in attributes:
    for y in attributes:
        if x != y:
            slope, intercept, r_value, p_value, std_err = linregress(
                daily_df_no_outliers[x], daily_df_no_outliers[y]
            )
            r2_values_daily[(x, y)] = r_value**2

r2_values_daily

In [ ]:
bad_sites = []

i = 2
site_name = combined_names[i]
site_type = combined_types[i]
print([site_name, site_type])
selected_ec = combined_ec[combined_ec["name"] == site_name]
selected_refl = combined_refl[combined_refl["name"] == site_name]
selected_fpar = combined_fpar[combined_fpar["name"] == site_name]

if selected_ec["PPFD_IN_QC"].isna().all():
    print("No PPFD data for this site")
    bad_sites.append(site_name)
    # continue

gpp = selected_ec[["GPP_NT_VUT_REF"]]
par = selected_ec[["PPFD_IN"]]
par_qc = selected_ec[["PPFD_IN_QC"]]
ec_daily = pd.concat([gpp, par, par_qc], axis=1).rename(
    columns={"GPP_NT_VUT_REF": "gpp", "PPFD_IN": "par", "PPFD_IN_QC": "par_qc"}
)
ec_daily = ec_daily[ec_daily["par_qc"] == 1]
ec_daily = ec_daily[ec_daily["gpp"] != -9999]
if len(ec_daily) < 30:
    print("Not enough data for this site")
    bad_sites.append(site_name)
    # continue

filtered_fpar = selected_fpar[
    (selected_fpar["MCD15A3H_061_FparLai_QC_MODLAND"] == "0b0")
    & (selected_fpar["MCD15A3H_061_FparLai_QC_DeadDetector"] == "0b0")
    & (selected_fpar["MCD15A3H_061_FparLai_QC_CloudState"] == "0b00")
    & (selected_fpar["MCD15A3H_061_FparLai_QC_SCF_QC"].isin(["0b000", "0b001"]))
].copy()

filtered_fpar.loc[:, "time"] = pd.to_datetime(filtered_fpar["Date"])
fpar_4days = filtered_fpar[["MCD15A3H_061_Fpar_500m"]]
fpar_4days.set_index(filtered_fpar["time"], inplace=True)
fpar_daily = fpar_4days.resample("D").interpolate(method="linear")
fpar_daily.rename(columns={"MCD15A3H_061_Fpar_500m": "fpar"}, inplace=True)

filtered_refl = selected_refl[
    (
        selected_refl["MCD43A4_061_BRDF_Albedo_Band_Mandatory_Quality_Band1_MODLAND"]
        == "0b000"
    )
    & (
        selected_refl["MCD43A4_061_BRDF_Albedo_Band_Mandatory_Quality_Band2_MODLAND"]
        == "0b000"
    )
].copy()
filtered_refl.loc[:, "time"] = pd.to_datetime(filtered_refl["Date"])
red_daily = filtered_refl[["MCD43A4_061_Nadir_Reflectance_Band1"]]
nir_daily = filtered_refl[["MCD43A4_061_Nadir_Reflectance_Band2"]]

refl_daily = pd.concat([red_daily, nir_daily], axis=1).rename(
    {
        "MCD43A4_061_Nadir_Reflectance_Band1": "red",
        "MCD43A4_061_Nadir_Reflectance_Band2": "nir",
    },
    axis=1,
)
refl_daily.set_index(filtered_refl["time"], inplace=True)
daily_df = ec_daily.merge(refl_daily, left_index=True, right_index=True).merge(
    fpar_daily, left_index=True, right_index=True
)

daily_df.loc[:, "ndvi"] = (daily_df["nir"] - daily_df["red"]) / (
    daily_df["nir"] + daily_df["red"]
)
daily_df.loc[:, "nirv"] = daily_df["ndvi"] * daily_df["nir"]
daily_df.loc[:, "nirvp"] = daily_df["nirv"] * daily_df["par"]
daily_df.loc[:, "fesc"] = daily_df["nirv"] / daily_df["fpar"]

daily_df.loc[:, "lue"] = daily_df["gpp"] / (daily_df["par"] * daily_df["fpar"])
attributes = ["lue", "fesc", "nirv", "nirvp", "ndvi", "gpp"]
scatter_matrix(daily_df_no_outliers[attributes], figsize=(10, 10))
r2_values_daily = {}
attributes = ["lue", "fesc", "nirv", "nirvp", "ndvi", "gpp"]

for x in attributes:
    for y in attributes:
        if x != y:
            slope, intercept, r_value, p_value, std_err = linregress(
                daily_df_no_outliers[x], daily_df_no_outliers[y]
            )
            r2_values_daily[(x, y)] = r_value**2

In [ ]:
from scipy.stats import linregress
from pandas.plotting import scatter_matrix
import pandas as pd


def process_site_data(
    i, combined_names, combined_types, combined_ec, combined_refl, combined_fpar
):
    bad_sites = []
    site_name = combined_names[i]
    site_type = combined_types[i]
    selected_ec = combined_ec[combined_ec["name"] == site_name]
    selected_refl = combined_refl[combined_refl["name"] == site_name]
    selected_fpar = combined_fpar[combined_fpar["name"] == site_name]

    if selected_ec["PPFD_IN_QC"].isna().all():
        print("No PPFD data for " + site_name)
        bad_sites.append(site_name)
        return (
            bad_sites,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
        )

    ec_daily = process_ec_data(selected_ec)
    if len(ec_daily) < 30:
        print("Not enough data for " + site_name)
        bad_sites.append(site_name)
        return (
            bad_sites,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
        )

    fpar_daily = process_fpar_data(selected_fpar)
    if len(fpar_daily) < 30:
        print("Not enough fpar for" + site_name)
        bad_sites.append(site_name)
        return (
            bad_sites,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
        )

    refl_daily = process_refl_data(selected_refl)
    if len(refl_daily) < 30:
        print("Not enough reflectance data for " + site_name)
        bad_sites.append(site_name)
        return (
            bad_sites,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
        )

    daily_df = merge_data(ec_daily, refl_daily, fpar_daily)
    if len(daily_df) < 30:
        print("Not enough data for " + site_name)
        bad_sites.append(site_name)
        return (
            bad_sites,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
        )
    daily_df = calculate_indices(daily_df)
    daily_df = daily_df.replace([-np.inf, np.inf], np.nan).dropna()
    daily_df_no_outliers = remove_outliers(
        daily_df
    )  # Assuming you have a way to remove outliers

    r2_values_daily, p_value_daily = calculate_r2_values(daily_df_no_outliers)

    weekly_df = daily_df_no_outliers.resample("W").mean()
    weekly_df_no_outliers = remove_outliers(weekly_df)
    weekly_df_no_outliers.dropna(inplace=True)
    r2_values_weekly, p_value_weekly = calculate_r2_values(weekly_df_no_outliers)

    monthly_df = daily_df_no_outliers.resample("M").mean()
    monthly_df_no_outliers = remove_outliers(monthly_df)
    monthly_df_no_outliers.dropna(inplace=True)
    r2_values_monthly, p_value_monthly = calculate_r2_values(monthly_df_no_outliers)

    return (
        bad_sites,
        r2_values_daily,
        p_value_daily,
        r2_values_weekly,
        p_value_weekly,
        r2_values_monthly,
        p_value_monthly,
        daily_df_no_outliers,
        weekly_df_no_outliers,
        monthly_df_no_outliers,
        site_type,
        site_name,
    )


def process_ec_data(selected_ec):
    gpp = selected_ec[["GPP_NT_VUT_REF"]]
    par = selected_ec[["PPFD_IN"]]
    par_qc = selected_ec[["PPFD_IN_QC"]]
    ec_daily = pd.concat([gpp, par, par_qc], axis=1).rename(
        columns={"GPP_NT_VUT_REF": "gpp", "PPFD_IN": "par", "PPFD_IN_QC": "par_qc"}
    )
    ec_daily = ec_daily[ec_daily["par_qc"] == 1]
    ec_daily = ec_daily[ec_daily["gpp"] != -9999]
    return ec_daily


def process_fpar_data(selected_fpar):
    # Your code here
    # filter fpar based on QC flags
    filtered_fpar = selected_fpar[
        (selected_fpar["MCD15A3H_061_FparLai_QC_MODLAND"] == "0b0")
        & (selected_fpar["MCD15A3H_061_FparLai_QC_DeadDetector"] == "0b0")
        & (selected_fpar["MCD15A3H_061_FparLai_QC_CloudState"] == "0b00")
        & (selected_fpar["MCD15A3H_061_FparLai_QC_SCF_QC"].isin(["0b000", "0b001"]))
    ].copy()

    filtered_fpar.loc[:, "time"] = pd.to_datetime(filtered_fpar["Date"])
    fpar_4days = filtered_fpar[["MCD15A3H_061_Fpar_500m"]]
    fpar_4days.set_index(filtered_fpar["time"], inplace=True)
    fpar_daily = fpar_4days.resample("D").interpolate(method="linear")
    fpar_daily.rename(columns={"MCD15A3H_061_Fpar_500m": "fpar"}, inplace=True)
    return fpar_daily


def process_refl_data(selected_refl):
    # Your code here
    filtered_refl = selected_refl[
        (
            selected_refl[
                "MCD43A4_061_BRDF_Albedo_Band_Mandatory_Quality_Band1_MODLAND"
            ]
            == "0b000"
        )
        & (
            selected_refl[
                "MCD43A4_061_BRDF_Albedo_Band_Mandatory_Quality_Band2_MODLAND"
            ]
            == "0b000"
        )
    ].copy()
    filtered_refl.loc[:, "time"] = pd.to_datetime(filtered_refl["Date"])
    red_daily = filtered_refl[["MCD43A4_061_Nadir_Reflectance_Band1"]]
    nir_daily = filtered_refl[["MCD43A4_061_Nadir_Reflectance_Band2"]]

    refl_daily = pd.concat([red_daily, nir_daily], axis=1).rename(
        {
            "MCD43A4_061_Nadir_Reflectance_Band1": "red",
            "MCD43A4_061_Nadir_Reflectance_Band2": "nir",
        },
        axis=1,
    )
    refl_daily.set_index(filtered_refl["time"], inplace=True)
    return refl_daily


def merge_data(ec_daily, refl_daily, fpar_daily):
    # Your code here
    daily_df = ec_daily.merge(refl_daily, left_index=True, right_index=True).merge(
        fpar_daily, left_index=True, right_index=True
    )
    return daily_df


def calculate_indices(daily_df):
    # w_nir = 0.9789
    # w_green = 0.4898
    daily_df.loc[:, "ndvi"] = (daily_df["nir"] - daily_df["red"]) / (
        daily_df["nir"] + daily_df["red"]
    )
    daily_df.loc[:, "nirv"] = daily_df["ndvi"] * daily_df["nir"]
    daily_df.loc[:, "nirvp"] = daily_df["nirv"] * daily_df["par"]
    daily_df.loc[:, "fesc"] = daily_df["nirv"] / daily_df["fpar"]
    # daily_df.loc[:, "p"] = (
    #     (daily_df["nir"] / w_nir) - (daily_df["green"] / w_green)
    # ) / (daily_df["nir"] - daily_df["green"])
    daily_df.loc[:, "lue"] = daily_df["gpp"] / (daily_df["par"] * daily_df["fpar"])
    return daily_df


def remove_outliers(df):
    # Your code here
    z_scores = np.abs(zscore(df))

    # Set a threshold for outliers
    threshold = 2

    # Get a boolean mask where True indicates it is an outlier
    outliers = (z_scores > threshold).any(axis=1)

    # Remove outliers
    df_no_outliers = df[~outliers].copy()
    return df_no_outliers


def calculate_r2_values(df):
    r2_values = {}
    p_values = {}
    attributes = ["lue", "fesc", "nirv", "nirvp", "ndvi", "gpp"]
    for x in attributes:
        for y in attributes:
            if x != y:
                slope, intercept, r_value, p_value, std_err = linregress(df[x], df[y])
                r2_values[(x, y)] = r_value**2
                p_values[(x, y)] = p_value
    return (r2_values, p_value)

In [ ]:
i = 106  # US-Ha1
# i = 186
(
    bad_sites,
    r2_values_daily,
    p_value_daily,
    r2_values_weekly,
    p_value_weekly,
    r2_values_monthly,
    p_value_monthly,
    daily_df_no_outliers,
    weekly_df_no_outliers,
    monthly_df_no_outliers,
    site_type,
    site_name,
) = process_site_data(
    i, combined_names, combined_types, combined_ec, combined_refl, combined_fpar
)
site_name, r2_values_monthly

In [ ]:
bad_sites_list = []
good_sites_data = []

# Loop through sites
for i in range(len(combined_names)):
    print(i)
    (
        bad_sites,
        r2_values_daily,
        p_value_daily,
        r2_values_weekly,
        p_value_weekly,
        r2_values_monthly,
        p_value_monthly,
        daily_df_no_outliers,
        weekly_df_no_outliers,
        monthly_df_no_outliers,
        site_type,
        site_name,
    ) = process_site_data(
        i, combined_names, combined_types, combined_ec, combined_refl, combined_fpar
    )

    if len(bad_sites) > 0:
        bad_sites_list.extend(bad_sites)
    else:
        good_sites_data.append(
            {
                "Site Name": site_name,
                "Site Type": site_type,
                "R2 Values daily": r2_values_daily,
                "p Values daily": p_value_daily,
                "R2 Values weekly": r2_values_weekly,
                "p Values weekly": p_value_weekly,
                "R2 Values monthly": r2_values_monthly,
                "p Values monthly": p_value_monthly,
                "daily_df_no_outliers": daily_df_no_outliers,
                "weekly_df_no_outliers": weekly_df_no_outliers,
                "monthly_df_no_outliers": monthly_df_no_outliers,
            }
        )

In [ ]:
import seaborn as sns

# Create a DataFrame from good_sites_data
df = pd.DataFrame(good_sites_data)

# Extract R2 values into a separate DataFrame
r2_df = pd.DataFrame(df["R2 Values weekly"].tolist())
r2_df["Site Name"] = df["Site Name"]
r2_df["Site Type"] = df["Site Type"]
# Melt the DataFrame to a long format
melted_df = r2_df.melt(
    id_vars=["Site Name", "Site Type"], var_name="Pair", value_name="R2"
)
selected_pairs = [("lue", "fesc"), ("lue", "nirv"), ("lue", "nirvp")]
filtered_df = melted_df[melted_df["Pair"].isin(selected_pairs)]
sns.boxplot(data=filtered_df, x="Site Type", y="R2", hue="Pair")
plt.savefig("../outputs/all_sites_weekly_r2.png")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a DataFrame from good_sites_data
df = pd.DataFrame(good_sites_data)

# Extract R2 values into a separate DataFrame
r2_df = pd.DataFrame(df["R2 Values weekly"].tolist())
r2_df["Site Name"] = df["Site Name"]
r2_df["Site Type"] = df["Site Type"]

# Melt the DataFrame to a long format
melted_df = r2_df.melt(
    id_vars=["Site Name", "Site Type"], var_name="Pair", value_name="R2"
)
selected_pairs = [("lue", "fesc"), ("lue", "nirv"), ("lue", "nirvp")]
filtered_df = melted_df[melted_df["Pair"].isin(selected_pairs)]

# Create boxplot
ax = sns.boxplot(data=filtered_df, x="Site Type", y="R2", hue="Pair")

# Annotate number of samples in each boxplot
for i, site_type in enumerate(filtered_df["Site Type"].unique()):
    # Select one of the pairs, for example "lue", "fesc"
    pair = selected_pairs[0]
    num_samples = len(
        filtered_df[
            (filtered_df["Site Type"] == site_type) & (filtered_df["Pair"] == pair)
        ]
    )
    max_y = filtered_df[
        (filtered_df["Site Type"] == site_type) & (filtered_df["Pair"] == pair)
    ]["R2"].max()
    ax.text(
        i,
        max_y + 0.01,
        f"n={num_samples}",
        horizontalalignment="center",
        size="small",
        color="black",
        weight="semibold",
    )

plt.savefig("../outputs/all_sites_weekly_r2.png")

In [ ]:
# Concatenate all 'daily_df_no_outliers' DataFrames into one
all_daily_df = pd.concat(
    [
        df.loc[i, "daily_df_no_outliers"].assign(site_type=df.loc[i, "Site Type"])
        for i in df.index
    ]
)
print("Number of daily data points:", len(all_daily_df))
print("Number of sites:", len(df))

In [ ]:
from scipy.stats import zscore

all_daily_df_no_outlier = all_daily_df[
    (np.abs(zscore(all_daily_df["lue"])) < 2) | (all_daily_df["lue"].isna())
]
all_daily_df_no_outlier

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
ax = sns.boxplot(data=all_daily_df_no_outlier, x="site_type", y="fesc")

# Annotate number of samples in each boxplot
for i, site_type in enumerate(all_daily_df_no_outlier["site_type"].unique()):
    num_samples = len(
        all_daily_df_no_outlier[all_daily_df_no_outlier["site_type"] == site_type]
    )
    max_y = all_daily_df_no_outlier[all_daily_df_no_outlier["site_type"] == site_type][
        "lue"
    ].max()
    ax.text(
        i,
        max_y - 0.01,
        f"n={num_samples}",
        horizontalalignment="center",
        size="small",
        color="black",
        weight="semibold",
    )

plt.title("Boxplot of LUE for each Site Type")
plt.savefig("../outputs/lue_boxplot.png")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
ax = sns.boxplot(data=all_daily_df_no_outlier, x="site_type", y="fpar")

# # Annotate number of samples in each boxplot
# for i, site_type in enumerate(all_daily_df_no_outlier["site_type"].unique()):
#     num_samples = len(all_daily_df_no_outlier[all_daily_df_no_outlier["site_type"] == site_type])
#     max_y = all_daily_df_no_outlier[all_daily_df_no_outlier["site_type"] == site_type]["fpar"].max()
#     ax.text(i, max_y - 0.01, f'n={num_samples}', horizontalalignment='center', size='small', color='black', weight='semibold')

plt.title("Boxplot of fpar for each Site Type")
plt.savefig("../outputs/fpar_boxplot.png")

In [ ]:
# Create dictionaries mapping site names to latitude and longitude
ameriflux_dict = ameriflux_info.set_index("Site ID")[
    ["Latitude (degrees)", "Longitude (degrees)"]
].to_dict(orient="index")
fluxnet_dict = fluxnet_info.set_index("ID")[["lat", "lon"]].to_dict(orient="index")

# Initialize an empty list to store the latitude and longitude values
lat_lon_list = []

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Extract site name from the row
    site_name = row["Site Name"]

    # Get latitude and longitude from ameriflux_dict if it exists, otherwise get it from fluxnet_dict
    if site_name in ameriflux_dict:
        lat = ameriflux_dict[site_name]["Latitude (degrees)"]
        lon = ameriflux_dict[site_name]["Longitude (degrees)"]
    elif site_name in fluxnet_dict:
        lat = fluxnet_dict[site_name]["lat"]
        lon = fluxnet_dict[site_name]["lon"]
    else:
        lat = None
        lon = None

    # Append the latitude and longitude to the list
    lat_lon_list.append({"Site Name": site_name, "Latitude": lat, "Longitude": lon})

# Create a DataFrame from the list of latitude and longitude values
lat_lon_df = pd.DataFrame(lat_lon_list)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Convert DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(lat_lon_df["Longitude"], lat_lon_df["Latitude"])]
geo_df = gpd.GeoDataFrame(lat_lon_df, geometry=geometry)

# Set the GeoDataFrame's coordinate system to geographic (EPSG:4326)
geo_df.set_crs("EPSG:4326", inplace=True)

# Save the GeoDataFrame as a GeoJSON file
geo_df.to_file("../outputs/lat_lon.geojson", driver="GeoJSON")